# Análise de Texto

O Processamento de Linguagem Natural (PLN) é um ramo da inteligência artificial (IA) que lida com a linguagem escrita e falada. Você pode usar o PLN para criar soluções que extraem o significado semântico de textos e falas ou formulam respostas significativas em linguagem natural.

Os *Serviços Cognitivos* do Microsoft Azure incluem o serviço de *Análise de Texto*, que oferece alguns recursos de PLN prontos para usar, inclusive a identificação de frases-chave no texto e a classificação de textos com base em sentimentos.

![Um robô lendo um notebook](./images/NLP.jpg)

Por exemplo, suponha que a empresa fictícia *Margie’s Travel* incentive os clientes a enviar análises de suas estadias em hotéis. Você pode usar o serviço de Análise de Texto para resumir as análises extraindo frases-chave, determinar quais delas são positivas e quais são negativas ou examinar o texto da análise em busca de menções a entidades conhecidas, como locais ou pessoas.

## Ver documentos de análise

Vamos começar dando uma olhada em algumas análises de hotéis que foram deixadas pelos clientes.

As análises estão em arquivos de texto. Para vê-las, basta executar o código abaixo clicando no botão **Executar célula** (&#9655;) à esquerda.

In [2]:
import os

# Ler as análises da pasta /data/reviews
reviews_folder = os.path.join('data', 'text', 'reviews')

# Criar uma coleção de análises com propriedades de ID (nome do arquivo) e texto (conteúdo)
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

review1.txt
Good Hotel and staff
The Royal Hotel, London, UK
3/2/2018
Clean rooms, good service, great location near Buckingham Palace and Westminster Abbey, and so on. We thoroughly enjoyed our stay. The courtyard is very peaceful and we went to a restaurant which is part of the same group and is Indian ( West coast so plenty of fish) with a Michelin Star. We had the taster menu which was fabulous. The rooms were very well appointed with a kitchen, lounge, bedroom and enormous bathroom. Thoroughly recommended.

review2.txt
Tired hotel with poor service
The Royal Hotel, London, United Kingdom
5/6/2018
This is a old hotel (has been around since 1950's) and the room furnishings are average - becoming a bit old now and require changing. The internet didn't work and had to come to one of their office rooms to check in for my flight home. The website says it's close to the British Museum, but it's too far to walk.

review3.txt
Good location and helpful staff, but on a busy road.
The Lombard

## Criar um recurso dos Serviços Cognitivos

Para examinar o texto dessas análises, use o serviço cognitivo de **Análise de Texto**. Para isso, é preciso provisionar um recurso de **Análise de Texto** ou **Serviços Cognitivos** na sua assinatura do Azure. (Use um recurso de Análise de Texto se esse for o único serviço que você pretende usar ou se quiser acompanhar a utilização separadamente. Caso contrário, poderá usar um recurso dos Serviços Cognitivos para combinar o serviço de Análise de Texto com outros serviços cognitivos, o que permite que os desenvolvedores usem um único ponto de extremidade e chave para acessá-los).

Caso você ainda não tenha um, use as etapas a seguir para criar um recurso dos **Serviços Cognitivos** na sua assinatura do Azure:

> **Observação**: Se você já tiver um recurso dos Serviços Cognitivos, abra a página de **Início Rápido** no portal do Azure e copie a respectiva chave e o ponto de extremidade para a célula abaixo. Caso contrário, siga as etapas abaixo para criar um.

1. Em outra guia do navegador, abra o portal do Azure em https://portal.azure.com, entrando com sua conta Microsoft.
2. Clique no botão **&#65291;Criar um recurso**, procure *Serviços Cognitivos* e crie um recurso dos **Serviços Cognitivos** com as configurações abaixo:
    - **Assinatura**: *sua assinatura do Azure*.
    - **Grupo de recursos**: *Selecione ou crie um grupo de recursos com um nome exclusivo*.
    - **Região**: *Escolha qualquer região disponível*:
    - **Nome**: *Insira um nome exclusivo*.
    - **Tipo de preço**: S0
    - **Confirmo que li e entendi os avisos**: Selecionado.
3. Aguarde até que a implantação seja concluída. Depois, entre em seu recurso dos Serviços Cognitivos e, na página **Visão geral**, clique no link para gerenciar as chaves do serviço. Você precisará do ponto de extremidade e das chaves para se conectar aos seus recursos dos Serviços Cognitivos em aplicativos clientes.

### Obter a chave e o ponto de extremidade do seu recurso dos Serviços Cognitivos

Para usar seu recurso dos Serviços Cognitivos, os aplicativos clientes precisam do ponto de extremidade e da chave de autenticação:

1. No portal do Azure, na página **Chaves e ponto de extremidade** do seu recurso dos Serviços Cognitivos, copie a **Chave 1** do recurso e cole no código abaixo, substituindo **YOUR_COG_KEY**.
2. Copie o **ponto de extremidade** do recurso e cole no código abaixo, substituindo **YOUR_COG_ENDPOINT**.
3. Execute o código da célula abaixo clicando no botão verde <span style="color:green">&#9655;</span>.

In [ ]:
cog_key = 'e09c79f3eeb54c2894df8b52f297e8f2'
cog_endpoint = 'https://cs-hsouza-01.cognitiveservices.azure.com/'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Detectar idioma
Vamos começar identificando o idioma em que essas análises foram escritas.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Obter um cliente para seu recurso do serviço cognitivo de Análise de Texto
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Examinar as análises que você leu na pasta /data/reviews anteriormente
language_analysis = text_analytics_client.detect_language(documents=reviews)

# imprimir detalhes da linguagem detectada em cada análise
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Extrair frases-chave

Agora você já pode examinar o texto das análises dos clientes para identificar frases-chave que indicam os principais pontos discutidos.

In [ ]:
# # Usar os clientes e as análises criadas na célula de código anterior para extrair frases-chave
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# imprimir frases-chave de cada análise
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

As frases-chave podem ajudar você a entender os principais pontos de discussão de cada análise. Por exemplo, uma análise que contenha a frase "equipe prestativa" ou "atendimento ruim" pode dar pistas sobre algumas das principais preocupações do avaliador.

## Determinar o sentimento

Talvez seja útil classificar as análises como *positivas* ou *negativas* de acordo com uma *pontuação de sentimento*. De novo, você pode usar o serviço de Análise de Texto para isso.

In [ ]:
# Usar os clientes e as análises que você criou anteriormente para obter pontuações de sentimento
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Imprimir os resultados de cada análise
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Extrair entidades conhecidas

*Entidades* são coisas que podem ser mencionadas no texto e fazem referência a algum tipo de item de compreensão comum. Por exemplo, um local, uma pessoa ou uma data. Vamos supor que você esteja interessado em datas e locais mencionados nas análises. Você poderá usar o código abaixo para identificá-los.

In [ ]:
# Usar os clientes e as análises que você criou anteriormente para obter entidades nomeadas
entity_analysis = text_analytics_client.entities(documents=reviews)

# Imprimir os resultados de cada análise
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only get location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Note que algumas entidades são conhecidas o bastante para ter uma página da Wikipedia associada a elas. Nesses casos, o serviço de Análise de Texto retorna o URL dessa página.

## Saiba mais

Para mais informações sobre o serviço de Análise de Texto, consulte a [documentação do serviço](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)